In [1]:
import pandas as pd
import numpy as np

In [2]:
df_log=pd.read_csv("all_log.csv", dtype=object)

In [3]:
df_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63668283 entries, 0 to 63668282
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   time         object
 1   user_id      object
 2   creative_id  object
 3   click_times  object
dtypes: object(4)
memory usage: 1.9+ GB


In [6]:
sentences = df_log.groupby(['user_id']).apply(lambda x: x['creative_id'].tolist()).tolist()

In [2]:
!pip install gensim
from gensim.models import Word2Vec

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [8]:
import time
start = time.clock()
model = Word2Vec(sentences=sentences, min_count=1, sg=1, window=10, size=32, workers=25, seed=2020, iter=5)
end = time.clock()
print('Running time: %s Seconds'%(end-start))
model.save("Word2vec.w2v")

Running time: 8526.568585 Seconds


In [3]:
model = Word2Vec.load("Word2vec.w2v").wv

In [28]:
# train_X = np.array(df_log.groupby(['user_id']).apply(lambda x: model[x['creative_id']]))

In [7]:
train=pd.read_csv("train_all.csv")

In [30]:
import sys,time
user_list=[]
for n in range(0,900000):
    sum=0
    item_list=train["user_items"].loc[n].split(" ")
    for i in item_list:
        sum+=model[i]
    user_list.append((sum/len(item_list)).tolist())
    
    sys.stdout.write('\r%s%%'%((n+1)*100/900000))
    sys.stdout.flush()

100.0%88888888889%%%%%%

In [31]:
em=pd.DataFrame(user_list)
em

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-1.076097,-0.413778,-0.149624,0.164099,-0.616197,0.633794,0.052338,0.699668,0.594100,0.546349,...,0.159658,-0.711443,-0.304250,-0.459857,0.106985,-0.147017,0.201753,-0.063981,-0.104358,-0.875712
1,-0.561944,-0.882193,-0.470752,0.247936,0.006558,0.647736,-0.125175,0.955615,0.356407,0.065717,...,0.380888,-0.221938,-0.505734,-0.408987,-0.155086,-0.032023,0.051463,-0.182851,0.039500,-0.625187
2,-0.888129,-0.144940,-0.135400,0.204846,-0.057031,1.087156,0.002011,0.303990,0.314979,-0.079577,...,-0.146939,-0.699824,-0.721158,-0.005146,0.086127,-0.047992,0.208833,-0.145681,0.662119,-0.706544
3,-0.795792,-0.557741,-0.200200,0.350504,0.110895,0.828112,-0.300809,0.493024,-0.110576,-0.036700,...,-0.191897,-0.365744,-0.739527,-0.244027,-0.071203,-0.221822,0.029207,-0.354225,-0.076688,-0.876255
4,-1.252417,-0.466689,0.072368,0.458773,-0.506256,0.460172,0.244700,0.189255,0.688625,0.234088,...,-0.194496,-0.539763,-0.186120,0.063950,-0.195155,-0.309231,0.370481,-0.273698,0.104554,-0.541433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,-0.715120,-0.090778,0.256138,0.364367,-0.036616,0.662755,-0.107509,0.823507,0.694637,0.500129,...,-0.148235,-0.492359,-0.258766,-0.252961,-0.108325,-0.251800,0.412047,0.061267,0.022834,-0.772559
899996,-0.619671,-0.426256,0.230679,0.697953,-0.241907,0.744501,0.249840,0.655470,0.294811,-0.136676,...,0.702625,0.005961,-0.487769,-0.049642,0.193844,0.070089,0.270810,0.356869,0.291610,-1.112071
899997,-0.428538,-0.537890,0.098799,0.382980,-0.392430,1.287879,0.193719,0.146630,-0.113477,0.452855,...,0.768047,-0.871338,-0.389162,0.075150,0.251305,-0.101896,0.168505,0.105804,0.795559,-2.089292
899998,-1.116559,-0.648528,-0.026307,0.386125,-0.511883,0.505082,-0.183182,0.637166,0.233982,-0.128932,...,0.304152,-0.518675,-0.345766,-0.326577,0.220268,0.141603,0.319794,0.139695,-0.078937,-0.691308


In [32]:
em.to_csv("w2v_train.csv",index=False)

In [4]:
test=pd.read_csv("test_all.csv")

In [ ]:
import sys,time
user_list=[]
for n in range(0,1000000):
    sum=0
    item_list=test["user_items"].loc[n].split(" ")
    for i in item_list:
        sum+=model[i]
    user_list.append((sum/len(item_list)).tolist())
    
    sys.stdout.write('\r%s%%'%((n+1)*100/1000000))
    sys.stdout.flush()

99.994%%

In [6]:
em=pd.DataFrame(user_list)
em

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-1.001465,-0.504781,0.558055,0.242638,0.158670,0.849774,-0.205765,1.072058,0.824865,0.717050,...,0.094894,-0.602741,0.064840,0.422890,-0.225110,-0.517799,0.201053,-0.484101,0.340232,-1.061414
1,-1.040575,-0.204014,-0.034565,0.728311,-0.471390,0.747581,0.146347,0.523329,0.215329,-0.275419,...,0.261692,-0.228608,-0.712106,-0.159497,0.237398,0.026463,0.380630,0.102838,0.027343,-0.683222
2,-0.654973,-0.878496,-0.224941,0.471162,-0.033702,0.749319,-0.022179,0.735424,0.140773,0.209458,...,0.347693,-0.177793,-0.258133,-0.203406,0.135398,0.117216,0.161895,-0.234810,0.188811,-0.833569
3,-0.658327,-0.678646,-0.188619,0.381383,0.124916,0.606880,0.047816,0.831569,0.733997,-0.000161,...,0.485413,-0.274927,-0.502537,-0.120699,-0.119815,-0.177241,0.143980,-0.160984,0.043389,-0.500303
4,-1.134198,-0.001912,-0.222266,0.556737,-0.063381,0.600158,0.067305,0.392954,0.804473,0.236884,...,-0.034810,-0.637644,-0.166306,0.046467,0.386256,-0.151892,0.267453,-0.350364,0.049397,-0.641194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,-0.521366,-0.015541,-0.208141,0.138651,0.061644,0.942160,-0.029351,0.517416,1.044086,0.555203,...,-0.113983,-0.662313,-0.375905,0.009173,0.222937,-0.304423,0.066290,-0.358775,0.206643,-0.563460
999996,-1.053769,-0.251412,0.200376,0.406260,-0.477592,0.507057,0.085747,0.525145,0.564927,0.295225,...,0.154024,-0.363251,-0.065023,-0.177442,-0.178469,-0.243985,0.118201,-0.448477,-0.107322,-0.587106
999997,-1.062344,-0.166667,0.007906,0.685657,-0.461282,0.446009,0.152632,0.357006,0.784723,0.298746,...,0.350711,-0.268540,-0.119366,0.157000,-0.183376,-0.265546,0.234567,-0.761166,-0.440231,-0.706164
999998,-0.861608,-0.343486,0.150691,0.319839,-0.194359,0.498770,0.220318,0.406327,0.562152,0.255614,...,-0.070215,-0.333924,-0.139270,-0.097743,-0.114120,-0.579649,0.289354,-0.548226,-0.107326,-0.591100


In [7]:
em.to_csv("w2v_test.csv",index=False)